In [2]:
import pandas as pd
from docx import Document


In [12]:
network_df = pd.read_excel("reseau_en_arbre.xlsx")


broken_network_df = network_df[network_df["infra_type"] == "a_remplacer"]


set_id_batiments = set(network_df["id_batiment"].values)
set_id_broken_batiments = set(broken_network_df["id_batiment"].values)

list_id_batiment = []
state_batiment = []

for id_batiment in set_id_batiments:
    list_id_batiment.append(id_batiment)
    if id_batiment in set_id_broken_batiments:
        state_batiment.append("à_reparer")  # bâtiment à réparer
    else:
        state_batiment.append("intact")     # bâtiment intact


state_df = pd.DataFrame({
    "id_bat": list_id_batiment,
    "state_batiment": state_batiment
})

header_row = pd.DataFrame([state_df.columns], columns=state_df.columns)

df_final = pd.concat([header_row, state_df], ignore_index=True)

df_final.to_excel("etat_batiment.xlsx", index=False, header=False)
print("Fichier 'etat_batiment.xlsx' généré avec succès.")


Fichier 'etat_batiment.xlsx' généré avec succès.


1- Il faut recupérer pour un seul infra les differents batiment qu'il permet de raccorder
2- la metrique d'évaluation de la priorité pour le moment est longueur_d'infra/nbr_maison(nbr_prise) métrique de difficulté pour reconstruire un seul infra.
3- pour chaque batiment on a une liste des infras du départ jusqu'au arrivée (de l'origine de l'infra jusqu'au dernier batiment), c'est déjà ordonnée dans le fichier xlsx
4- résultat : 4 phases de reconstruction (4 degrées de priorité pour chaque batiment)
5- il faut résonner en fonction d'un chemin d'infra (et pas par 1 infra parce que ça sert à rien de reconstruire une infra dans la 1ere phase par exemple sans reconstituer l'electricité dans le batiment). du coup la métrique devient : somme(longueur_d'infra/nbr_maison_dependant_de_cette_infra) pour chaque batiment. Exemple :
id_batiment	nb_maisons	infra_id	infra_type	longueur
E000001	4	P007111	infra_intacte	12.3144613356693
E000001	4	P007983	infra_intacte	40.3209288665353
E000001	4	P000308	infra_intacte	39.1407985664577
E000001	4	P007819	infra_intacte	17.3904643240185
6- on va calculer la difficulté de chaque infra , pour faire la somme facilement par batiment.
7- quand on répare un infra il faut mettre à jour la métrique.
8- metrique final cout*duree/nbr_prise par batiment = somme de cout*duree/nbr_prise par infra
9- on va tout simplement dans le tableau excel ajouter une colonne cout en euros et une colonne durée en temps et une colonne type_reparation(aerien/semi-aérien/fourreau)
10- pour la mise a jour c'est simple , a chaque fois qu'une infra est remplacé on la fait sortir de la liste des broken_infra et on recalcule la somme de cout*duree/nbr_prise par infra .


In [3]:
class Infra:

    def __init__(self,infra_id,length,infra_type,nb_house):
        self.infra_id = infra_id
        self.length = length
        self.infra_type = infra_type
        self.nb_house = nb_house

    def __repr__(self):
        return f"Infrastructure {self.infra_id} | Type: {self.infra_type} | Length: {self.length}m | Houses: {self.nb_house}"

    def repair_infra(self):
        pass

    def get_infra_difficulty(self) :
        return self.length/self.nb_house
    

In [6]:
class Building:

    def __init__(self,id_building,list_infra):
        self.id_building = id_building
        self.list_infra = []

    def add_infra(self, infra):
        self.list_infra.append(infra)

    def get_building_difficulty(self) :
        s = 0
        for infra in self.list_infra : s+=infra.get_infra_difficulty()
        return s

    def add_infra(self, infra):
        self.list_infra.append(infra)
        
    def __repr__(self):
        return f"Building {self.id_building} | {self.list_infra}"


In [8]:
#1er script : calcul de la 1ere métrique :

network_df = pd.read_excel("reseau_en_arbre.xlsx")
broken_network_df = network_df[network_df["infra_type"] == "a_remplacer"]


broken_infra = []
buildings_without_electricity = []
infra_difficulties = {}
buildings = {}
building_difficulties = {}

for _,infra in broken_network_df.iterrows():
    infrastructure = Infra(infra_id=infra["infra_id"], length=infra["longueur"], infra_type=infra["infra_type"], nb_house=infra["nb_maisons"])
    broken_infra.append(infrastructure)

print(broken_infra)

for infra, building_id in zip(broken_infra, broken_network_df["id_batiment"]):
    if building_id not in buildings:
        buildings[building_id] = Building(building_id, [])
    buildings[building_id].list_infra.append(infra)

buildings_without_electricity = list(buildings.values())

print(buildings_without_electricity)

for infra in broken_infra :
    infra_difficulties[infra.infra_id] = infra.get_infra_difficulty()

print(infra_difficulties)

for building in buildings_without_electricity :
    building_difficulties[building.id_building] = building.get_building_difficulty()


final_result = dict(sorted(building_difficulties.items(), key=lambda item: item[1]))
print(final_result)




[Infrastructure P007113 | Type: a_remplacer | Length: 12.1935087083631m | Houses: 1, Infrastructure P007984 | Type: a_remplacer | Length: 30.0575979492587m | Houses: 1, Infrastructure P007823 | Type: a_remplacer | Length: 12.290282829583m | Houses: 1, Infrastructure P006193 | Type: a_remplacer | Length: 12.1863879405322m | Houses: 1, Infrastructure P002851 | Type: a_remplacer | Length: 10.8032015604234m | Houses: 1, Infrastructure P007624 | Type: a_remplacer | Length: 4.10990474598487m | Houses: 1, Infrastructure P005500 | Type: a_remplacer | Length: 13.1420349052071m | Houses: 1, Infrastructure P007990 | Type: a_remplacer | Length: 18.7028610321555m | Houses: 1, Infrastructure P007447 | Type: a_remplacer | Length: 2.84535154473363m | Houses: 1, Infrastructure P005531 | Type: a_remplacer | Length: 14.8493095953268m | Houses: 1, Infrastructure P007639 | Type: a_remplacer | Length: 6.13280316452202m | Houses: 1, Infrastructure P005500 | Type: a_remplacer | Length: 13.1420349052071m | Hou

In [9]:
#Redéfinition de la classe Infra
class Infra:

    def __init__(self,infra_id,length,infra_type,nb_house,cost,time):
        self.infra_id = infra_id
        self.length = length
        self.infra_type = infra_type
        self.nb_house = nb_house
        self.cost = cost
        self.time = time

    def __repr__(self):
        return f"Infrastructure {self.infra_id} | Type: {self.infra_type} | Length: {self.length}m | Houses: {self.nb_house} | cost: {self.cost} | time: {self.time}"

    def repair_infra(self) :
        pass

    def get_infra_difficulty(self) :
        return self.cost*self.time/self.nb_house

In [10]:
#2éme script : Calcul de la nouvelle métrique.

#Méthode de lecture et de normalisation du fichier annexe 
def extract_infra_costs_and_times(appendix):
    doc = Document(appendix)
    text = "\n".join([p.text for p in doc.paragraphs])
    
    prix_dict = {}
    duree_dict = {}
    current = None
    
    for line in text.splitlines():
        line = line.strip().lower()
        if not line:
            continue
        if "prix" in line:
            current = "prix"
            continue
        if "durée" in line or "duree" in line:
            current = "duree"
            continue

        if ":" in line:
            type_infra, val = line.split(":", 1)
            type_infra = type_infra.strip()
            val = val.strip().split(" ")[0]
            val_numeric = ''.join(c for c in val if c.isdigit() or c == ".")
            if not val_numeric:  # ignore si rien de numérique
                continue
            if current == "prix":
                prix_dict[type_infra] = float(val_numeric)
            elif current == "duree":
                duree_dict[type_infra] = float(val_numeric)
    
    return prix_dict, duree_dict


#Méthode regroupant tout les informations essentielles au calcul de la métrique :
def complete_data(buildings,infra,network,appendix) :


    buildings_csv = pd.read_csv(buildings)
    infra_csv = pd.read_csv(infra)
    network_xlsx = pd.read_excel(network)
    prix_dict, duree_dict = extract_infra_costs_and_times(appendix)

    
    #Corriger le nombre des maisons pour chaque batiment
    buildings_correction = pd.merge(network_xlsx, buildings_csv, on="id_batiment", how="left", suffixes=("", "_csv1"))
    buildings_correction["nb_maisons"] = buildings_correction["nb_maisons_csv1"].combine_first(buildings_correction["nb_maisons"])
    buildings_correction = buildings_correction.drop(columns=["nb_maisons_csv1"])


    #Ajouter la colonne type_infra pour chaque infra
    infra_type = pd.merge(buildings_correction, infra_csv[["id_infra", "type_infra"]], left_on="infra_id",right_on="id_infra",how="left")
    infra_type = infra_type.drop(columns=["id_infra"])

    #Ajouter les colonnes de coût et durée pour reconstruire chaque infra
    infra_type["cout_total"] = infra_type["type_infra"].str.lower().map(prix_dict) * infra_type["longueur"]
    infra_type["duree_totale"] = infra_type["type_infra"].str.lower().map(duree_dict) * infra_type["longueur"]
    complete_data = infra_type.drop_duplicates()


    return complete_data


#Méthode calculant la métrique de difficulté pour chaque batiment: 
def final_metric(complete_data):

    broken_infra = []
    buildings_without_electricity = []
    buildings = {}
    infra_difficulties = {}
    building_difficulties = {}


    complete_data = complete_data[complete_data["infra_type"] == "a_remplacer"]
    for _,infra in complete_data.iterrows():
        infrastructure = Infra(infra_id=infra["infra_id"], length=infra["longueur"], infra_type=infra["infra_type"], nb_house=infra["nb_maisons"], cost=infra["cout_total"]
                               ,time=infra["duree_totale"])
        broken_infra.append(infrastructure)
    for infra, building_id in zip(broken_infra, complete_data["id_batiment"]):
        if building_id not in buildings:
            buildings[building_id] = Building(id_building=building_id, list_infra=[])
        buildings[building_id].list_infra.append(infra)
    buildings_without_electricity = list(buildings
                                         .values())
    
    for infra in broken_infra :
        infra_difficulties[infra.infra_id] = infra.get_infra_difficulty()
    

    for building in buildings_without_electricity :
        building_difficulties[building.id_building] = building.get_building_difficulty()
    

    return building_difficulties



In [11]:
#Difficulté de chaque batiment en euros.heure/nbr_maisons
final_metric(complete_data("batiments.csv","infra.csv","reseau_en_arbre.xlsx","annexe.docx"))


{'E000003': 1203191.90113135,
 'E000067': 258826.25963841038,
 'E000085': 558942.2065968337,
 'E000098': 55678.96162919077,
 'E000106': 794680.5394446917,
 'E000107': 405470.92050417175,
 'E000108': 770133.5013706668,
 'E000109': 757193.1852590976,
 'E000110': 714287.4547208382,
 'E000111': 62290.77769626339,
 'E000112': 38379.68795337064,
 'E000127': 45011.279628416785,
 'E000128': 64577.08245465618,
 'E000129': 402443.33145386743,
 'E000144': 217159.97287004982,
 'E000145': 348981.28146090265,
 'E000146': 232316.28230686177,
 'E000160': 93547.70416614751,
 'E000161': 733185.9639399853,
 'E000174': 2062286.0152687405,
 'E000175': 1095949.1874081586,
 'E000176': 3682203.871114696,
 'E000177': 4083861.4791622562,
 'E000178': 932447.9544800178,
 'E000183': 2067361.9686363894,
 'E000184': 285724.5689732736,
 'E000185': 424045.08218127897,
 'E000188': 296403.5192045903,
 'E000193': 869761.1249993393,
 'E000194': 73214.61480584975,
 'E000195': 91518.2685073122,
 'E000196': 122024.3580097495